### Initialize

In [1]:
import numpy as np
import csv

N = 10 # Node 24
F = 4 # Flow 22
H = 4 # Time 6
SetD = np.array([8, 9]) # DU node 22, 23

MinLink = 4
MaxLink = 4

# make output file
path = f'test{MinLink}{MaxLink}_{N}_{F}_{H}.cnf.txt'
with open(path, mode='w') as outfile:
    outfile.write('')

### Random link

In [2]:
import random

Link = [[0 for i in range(N)] for j in range(N)]
num_link = np.zeros(N, dtype=int)

for i in range(N):
    while num_link[i] < MinLink:
        tmp_dst = random.randrange(N)
        if num_link[tmp_dst] < MaxLink:
            Link[i][tmp_dst] = 1
            num_link[i] += 1
            Link[tmp_dst][i] = 1
            num_link[tmp_dst] += 1

for d in range(len(SetD)):
    for j in range(N):
        Link[SetD[d]][j] = 0 # DU is destination

for i in range(N):
    Link[i][i] = 1 # i -> i is queuing

print(Link)
np.savetxt(f'link_{N}_{F}_{H}.txt', Link, delimiter=' ', fmt='%.0f')


[[1, 0, 0, 0, 1, 0, 1, 1, 0, 1], [0, 1, 0, 1, 1, 0, 1, 0, 1, 0], [0, 0, 1, 1, 1, 0, 0, 1, 0, 0], [0, 1, 1, 1, 0, 0, 0, 0, 0, 0], [1, 1, 1, 0, 1, 0, 1, 0, 0, 0], [0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 1, 0, 1, 0], [1, 0, 1, 0, 0, 0, 0, 1, 0, 1], [0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 1]]


In [3]:
# read link file

#with open('link.csv') as linkfile:
#    SetLstr = list(csv.reader(linkfile))
#    SetL = []
#    for i in range(len(SetLstr)):
#        SetL.append([int(SetLstr[i][j]) for j in range(len(SetLstr[i]))])
#print(SetL)


### A flow is at either one of the nodes at each time slot: 
$$ x_{f,h,1} \lor x_{f,h,2} \lor ... \lor x_{f,h,N} \forall f,h $$


In [4]:
for f in range(F):
    for h in range(H):
        tmp_node = np.ones(N, dtype=int)
        tmp_one = 1 + f*H*N + h*N
        for n in range(N):
            tmp_node[n] = tmp_one + n
        tmp_ln = ' '.join(map(str, tmp_node))
        with open(path, mode='a') as outfile:
            outfile.write(tmp_ln + ' 0\n')


### A flow cannot be at multiple nodes at the same time: 
$$ \lnot x_{f,h,i} \lor \lnot x_{f,h,j} \forall f,h,i,j $$


In [5]:
for f in range(F):
    for h in range(H):
        tmp_one = 1 + f*H*N + h*N
        for i in range(N):
            for j in range(i+1, N):
                tmp_i = - (tmp_one + i)
                tmp_j = - (tmp_one + j)
                with open(path, mode='a') as outfile:
                    outfile.write(str(tmp_i) + ' ' + str(tmp_j) + ' 0\n')


### A flow can be forwarded from i-th node to j-th node only if connected: 
$$ \lnot x_{f,h,i} \lor \lnot x_{f,h+1,j} \quad \forall f,h,i,j \quad s.t. \quad l_{i,j} = 0 $$


In [6]:
for f in range(F):
    for h in range(H-1):
        tmp_one = 1 + f*H*N + h*N
        for i in range(N):
            for j in range(N):
                tmp_i = - (tmp_one + i)
                tmp_j = - (tmp_one + N + j)
                if Link[i][j] == 0:
                    with open(path, mode='a') as outfile:
                        outfile.write(str(tmp_i) + ' ' + str(tmp_j) + ' 0\n')


### The source node of f-th flow at 0-th time slot: 
$$ x_{f,0,s_{f}} \quad \forall f $$


In [7]:
for f in range(F):
    tmp_one = 1 + f*H*N + 0*N
    tmp_sf = tmp_one + f # s_f = f
    with open(path, mode='a') as outfile:
        outfile.write(str(tmp_sf) + ' 0\n')


### A flow has reached either one of DUs at H-th time slot: 
$$ x_{f,H,1} \lor x_{f,H,2} \dots \lor x_{f,H,d} \lor \dots \lor x_{f,H,D} $$


In [8]:
for f in range(F):
    tmp_one = 1 + f*H*N + (H-1)*N
    for d in range(len(SetD)):
        tmp_d = tmp_one + SetD[d]
        with open(path, mode='a') as outfile:
            outfile.write(str(tmp_d) + ' ')
    with open(path, mode='a') as outfile:
        outfile.write('0\n')


### Only one flow can be forwarded by a link at the same time slot: 
$$ \lnot x_{f,h,i} \lor \lnot x_{f,h+1,j} \lor \lnot x_{g,h,i} \lor \lnot x_{g,h+1,j} \quad \forall f,g,h,i,j $$


In [9]:
for f in range(F):
    for g in range(f+1, F):
        for h in range(H-1):
            tmp_one_f = 1 + f*H*N + h*N
            tmp_one_g = 1 + g*H*N + h*N
            for i in range(N):
                for j in range(i+1, N):
                    tmp_i_f = - (tmp_one_f + i)
                    tmp_j_f = - (tmp_one_f + N + j)
                    tmp_i_g = - (tmp_one_g + i)
                    tmp_j_g = - (tmp_one_g + N + j)
                    with open(path, mode='a') as outfile:
                        outfile.write(str(tmp_i_f) + ' ' + str(tmp_j_f) + ' ' + str(tmp_i_g) + ' ' + str(tmp_j_g) + ' 0\n')
